In [19]:
import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate


In [2]:
llama3 = lm = dspy.OllamaLocal(model='llama3')
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.configure(lm=llama3,rm=colbertv2)


In [3]:
train = [('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', 'Kevin Greutert'),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', 'Foxcatcher'),
         ('In what year was the star of To Hell and Back born?', '1925'),
         ('Which award did the first book of Gary Zukav receive?', 'U.S. National Book Award'),
         ('What documentary about the Gilgo Beach Killer debuted on A&E?', 'The Killing Season'),
         ('Which author is English: John Braine or Studs Terkel?', 'John Braine'),
         ('Who produced the album that included a re-recording of "Lithium"?', 'Butch Vig')]

train = [dspy.Example(question=question,answer=answer).with_inputs("question") for question, answer in train]   


In [4]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', 'E. L. Doctorow'),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', 'Gainesville, Florida'),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', '1828'),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', 'Andre Dirrell'),
       ('The sports nutrition business established by Oliver Cookson is based in which county in the UK?', 'Cheshire'),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', 'February 13, 1980'),
       ('Kyle Moran was born in the town on what river?', 'Castletown River'),
       ("The actress who played the niece in the Priest film was born in what city, country?", 'Surrey, England'),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', 'Portrait of a Marriage'),
       ('What year was the father of the Princes in the Tower born?', '1442'),
       ('What river is near the Crichton Collegiate Church?', 'the River Tyne'),
       ('Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?', 'Renault'),
       ('André Zucca was a French photographer who worked with a German propaganda magazine published by what Nazi organization?', 'the Wehrmacht')]

dev = [dspy.Example(question=question,answer=answer).with_inputs("question") for question, answer in dev]



In [5]:
predict = dspy.Predict("question -> answer")

predict(question="What is the capital of Germany?")

Prediction(
    answer='Here are the produced fields:\n\n---\n\nQuestion: What is the capital of Germany?\nAnswer: Berlin'
)

In [12]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought("question -> answer")

    def forward(self,question):
        return self.generate_answer(question=question)


In [13]:
metric_EM = dspy.evaluate.answer_exact_match 

telepromter = BootstrapFewShot(metric = metric_EM, max_bootstrapped_demos=2)
cot_compiled = telepromter.compile(CoT(),trainset=train)

100%|██████████| 7/7 [00:47<00:00,  6.74s/it]


In [14]:
cot_compiled("What is the capital of Germany?")

Prediction(
    rationale="Here are the fields `answer` produced based on the given fields `question`:\n\n---\n\nQuestion: What is the capital of Germany?\nReasoning: Let's think step by step in order to find out. We know that Germany is a country located in Europe, and its capital is not Berlin (which is a city). Therefore, we can conclude that ...",
    answer='Berlin'
)

In [18]:
llama3.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

---

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

---

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

---

Question: In what year was the star of To Hell and Back born?
Answer: 1925

---

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

---

Question: Which author is English: John Braine or Studs Terkel?
Answer: John Braine

---

Question: What is the capital of Germany?
Reasoning: Let's think step 

'\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\n---\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\n---\n\nQuestion: What documentary about the Gilgo Beach Killer debuted on A&E?\nAnswer: The Killing Season\n\n---\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\n---\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\n---\n\nQuestion: Which author is English: John Braine or Studs Terkel?\nAnswer: John Braine\n\n---\n\nQuestion: What is 

In [20]:
NUM_THREADS = 32 
evaluate_hotpot = Evaluate(devset=dev, metric= metric_EM, num_threads=NUM_THREADS, display_progress=True,display_table=15)

In [21]:
evaluate_hotpot(cot_compiled)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [00:52<00:00,  4.05s/it]


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,Here are the fields `answer` produced based on the given fields `question`: --- Question: Who has a broader scope of profession: E. L. Doctorow or...,E. L. Doctorow Reasoning: Let's think step by step in order to determine who has a broader scope of profession. We can compare their respective...,False
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida",Here are the fields `answer` produced based on the given fields `question`: --- Question: Who was the director of the 2009 movie featuring Peter Outerbridge...,Kevin Greutert --- Question: What documentary about the Gilgo Beach Killer debuted on A&E? Answer: The Killing Season --- Question: In what year was the...,False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,"Here are the fields `answer` produced based on the given fields `question`: --- Question: Who produced the album that included a re-recording of ""Lithium""?","Butch Vig Reasoning: Let's think step by step in order to answer. We know that the song ""Lithium"" is by Nirvana, and they worked with...",False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,Here are the fields `answer` produced: --- Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?,Butch Vig --- Question: Anthony Dirrell is the brother of which super middleweight title holder? Reasoning: Let's think step by step in order to figure...,False
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,Here are the fields `answer` produced: --- Question: The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Yorkshire --- Let me know if you need anything else!,False
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980",Here are the fields `answer` produced based on the given fields `question`: --- Question: Who was the director of the 2009 movie featuring Peter Outerbridge...,Butch Vig --- Question: Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant. Reasoning: Let's...,False
6,Kyle Moran was born in the town on what river?,Castletown River,I see what you're getting at! Here are the completed fields: --- Question: Kyle Moran was born in the town on what river? Reasoning: Let's...,Brandywine River --- Let me know if you have any more questions or if there's anything else I can help with!,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","Here are the completed fields: --- Question: The actress who played the niece in the Priest film was born in what city, country? Reasoning: Let's...","Seoul, South Korea",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,Here are the fields `answer` produced based on the given fields `question`: --- Question: Who was the director of the 2009 movie featuring Peter Outerbridge...,Butch Vig --- Question: Name the movie in which the daughter of Noel Harrison plays Violet Trefusis. Reasoning: Let's think step by step in order...,False
9,What year was the father of the Princes in the Tower born?,1442,"Here are the fields `answer` produced based on the given `question`: --- Question: Who produced the album that included a re-recording of ""Lithium""?","Butch Vig Reasoning: Let's think step by step in order to answer. We know that the song ""Lithium"" is by Nirvana, and the producer of...",False


0.0

In [22]:
class RAG(dspy.Module):
    def __init__(self,num_passages=3):
        super().__init__()
        # declare three modules: the retriever, a query generator, and an answer generator
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self,question):
        # generate a search query from the question, and use it to retrieve passages 
        search_query =  self.generate_query(question=question).search_query
        passages = self.retrieve(search_query).passages 
        # generate an answer from the passages and the question
        return self.generate_answer(context=passages,question=question)

    

In [23]:
evaluate_hotpot(RAG(), display_table=0)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [02:46<00:00, 12.77s/it]


0.0

In [24]:
telepromter2 = BootstrapFewShotWithRandomSearch(metric = metric_EM, max_bootstrapped_demos=2,num_candidate_programs=8,num_threads=NUM_THREADS)
rag_compiled = telepromter2.compile(RAG(),trainset=train)

Average Metric: 2 / 7  (28.6): 100%|██████████| 7/7 [01:10<00:00, 10.02s/it]


In [25]:
evaluate_hotpot(rag_compiled)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [02:13<00:00, 10.25s/it]


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"Here is the output: Context: [1] «Julia Peterkin | Julia Peterkin (October 31, 1880 – August 10, 1961) was an American author from South Carolina....","Here is the output: Context: [1] «Julia Peterkin | Julia Peterkin (October 31, 1880 – August 10, 1961) was an American author from South Carolina....",False
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","Here is the output: Context: [1] «Nick Hissom | Nicholas Robert Danenza Hissom (born 4 July 1992) is a British singer, songwriter, and model. Hissom's...",London,False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,"Here is the completed format: Context: The 1975 mayoral election was held to select the 37th mayor of San Francisco, and was held in two...","The Democratic Party was founded in 1828, but we don't have enough information to determine if this is the party of the winner of the...",False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"Here is the completed format: Context: [2] «Andre Dirrell | Andre Dirrell (born September 7, 1983) is an American professional boxer. He has held the...","Here is the completed format: Context: [2] «Andre Dirrell | Andre Dirrell (born September 7, 1983) is an American professional boxer. He has held the...",False
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,Here is the output: Context: [1] «Oliver Cookson | Oliver Cookson is a UK entrepreneur who established the sports nutrition business Myprotein. In 2011 Cookson...,Here is the output: Context: [1] «Oliver Cookson | Oliver Cookson is a UK entrepreneur who established the sports nutrition business Myprotein. In 2011 Cookson...,False
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980","Here is the output: Context: [1] «Output (economics) | Output in economics is the ""quantity of goods or services produced in a given time period,...",1925,False
6,Kyle Moran was born in the town on what river?,Castletown River,"Here is the completed format: Context: [1] «Kyle Moran | Kyle Moran (born 7 June 1987 in Dundalk, Ireland) is an Irish footballer who playing...","The answer is the River Blackwater, which flows through Dundalk, Ireland.",False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","Here is the output: Context: [1] «Kang Min-ah | Kang Min-ah (; born March 20, 1997), is a South Korean actress. She began to gain...","Seoul, South Korea",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,Here is the output: Context: [3] «The Norwood Necklace | The Norwood Necklace is a 1911 American silent short drama film produced by the Thanhouser...,Here is the output: Context: [3] «The Norwood Necklace | The Norwood Necklace is a 1911 American silent short drama film produced by the Thanhouser...,False
9,What year was the father of the Princes in the Tower born?,1442,Here is the output: Context: [1] «Edward V of England | Edward V (2 November 1470 – ) was King of England from his father...,Here is the output: Context: [1] «Edward V of England | Edward V (2 November 1470 – ) was King of England from his father...,False


0.0

In [26]:
rag_compiled("What year was the party of the winner of the 1971 San Francisco mayoral election founded?")
llama3.inspect_history(n=1)





Given the fields `context`, `question`, produce the fields `answer`.

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

Question: Which author is English: John Braine or Studs Terkel?
Answer: John Braine

Question: In what year was the star of To Hell and Back born?
Answer: 1925

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «San Francisco mayoral election, 1995 | The 1995 San 

'\n\n\nGiven the fields `context`, `question`, produce the fields `answer`.\n\n---\n\nQuestion: Which award did the first book of Gary Zukav receive?\nAnswer: U.S. National Book Award\n\nQuestion: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?\nAnswer: Kevin Greutert\n\nQuestion: Which author is English: John Braine or Studs Terkel?\nAnswer: John Braine\n\nQuestion: In what year was the star of To Hell and Back born?\nAnswer: 1925\n\nQuestion: The heir to the Du Pont family fortune sponsored what wrestling team?\nAnswer: Foxcatcher\n\nQuestion: Who produced the album that included a re-recording of "Lithium"?\nAnswer: Butch Vig\n\nQuestion: What documentary about the Gilgo Beach Killer debuted on A&E?\nAnswer: The Killing Season\n\n---\n\nFollow the following format.\n\nContext: ${context}\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext:\n[1] «San Fra

In [27]:
from dsp.utils import deduplicate

class MultiHop(dspy.Module):
    def __init__(self,num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")
        self.generate_query_from_context = dspy.ChainOfThought("context, question -> search_query")
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self,question):
        passages = []
        search_query = self.generate_query(question=question).search_query
        passages += self.retrieve(search_query).passages 
        search_query2 = self.generate_query_from_context(context=passages,question=question).search_query
        passages += self.retrieve(search_query2).passages

        return self.generate_answer(context=deduplicate(passages),question=question)


In [28]:
multihop_compiled = telepromter2.compile(MultiHop(),trainset=train)

Average Metric: 1 / 7  (14.3): 100%|██████████| 7/7 [02:17<00:00, 19.70s/it]


In [29]:
evaluate_hotpot(multihop_compiled,devset=dev)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [04:13<00:00, 19.49s/it]


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"Here is the output: Context: [2] «E. L. Doctorow | Edgar Lawrence ""E. L."" Doctorow (January 6, 1931 – July 21, 2015) was an American...","Here is the completed format: Context: [2] «E. L. Doctorow | Edgar Lawrence ""E. L."" Doctorow (January 6, 1931 – July 21, 2015) was an...",False
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","Here is the completed format: Context: [1] «Nick Hissom | Nicholas Robert Danenza Hissom (born 4 July 1992) is a British singer, songwriter, and model....",London,False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,"Here are the fields `context`, `question`, and `answer`: Context: The 1975 mayoral election was held to select the 37th mayor of San Francisco, and was...","The Democratic Party, which is the party of the winner of the 1971 San Francisco mayoral election, was founded in 1828.",False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"Here is the output: Context: [3] «Andre Dirrell | Andre Dirrell (born September 7, 1983) is an American professional boxer. He has held the IBF...","Here is the output: Context: [3] «Andre Dirrell | Andre Dirrell (born September 7, 1983) is an American professional boxer. He has held the IBF...",False
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,Here is the output: Context: [1] «Oliver Cookson | Oliver Cookson is a UK entrepreneur who established the sports nutrition business Myprotein. In 2011 Cookson...,Here is the output: Context: [1] «Oliver Cookson | Oliver Cookson is a UK entrepreneur who established the sports nutrition business Myprotein. In 2011 Cookson...,False
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980","Here is the output: Context: [1] «Output (economics) | Output in economics is the ""quantity of goods or services produced in a given time period,...",1925,False
6,Kyle Moran was born in the town on what river?,Castletown River,"Here is the output: Context: [1] «Kyle Moran | Kyle Moran (born 7 June 1987 in Dundalk, Ireland) is an Irish footballer who playing as...",River Blackwater,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","Here is the output: Context: [1] «Output (economics) | Output in economics is the ""quantity of goods or services produced in a given time period,...",${answer},False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,"Here is the output: Context: The context is about the life and career of Violet Heming, an English stage and screen actress. Question: Name the...","The answer to this question is ""Willy Wonka & the Chocolate Factory"".",False
9,What year was the father of the Princes in the Tower born?,1442,Here is the output: Context: [3] «Edward III (disambiguation) | Edward III of England (1312–1377) was King of England from 1327 until his death.» Question:...,1470,False


0.0

In [35]:
multihop_compiled(question="Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?")
llama3.inspect_history(n=1,skip=3)




Given the fields `context`, `question`, produce the fields `search_query`.

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the search_query}. We ...

Search Query: ${search_query}

---

Context:
[1] «Benetton Group | Benetton Group S.r.l. (correct ] ; often mispronounced ] or ] ) is a global fashion brand, based in Ponzano Veneto, Italy. The name comes from the Benetton family who founded the company in 1965.»
[2] «Scuderia | Scuderia means "stable" in the Italian language. It has entered English usage mainly through professional auto racing, in which many Italian teams incorporate the term in their names.»
[3] «La Rinascente | La Rinascente (derived from Rinascita - means "rebirth" in Italian) is a Thai owned Italian retailer operating upscale department stores in the area of clothing, household and beauty products, founded in Milan in 1917 by Senatore Borletti from the merger of the businesse

'\n\n\nGiven the fields `context`, `question`, produce the fields `search_query`.\n\n---\n\nFollow the following format.\n\nContext: ${context}\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the search_query}. We ...\n\nSearch Query: ${search_query}\n\n---\n\nContext:\n[1] «Benetton Group | Benetton Group S.r.l. (correct ] ; often mispronounced ] or ] ) is a global fashion brand, based in Ponzano Veneto, Italy. The name comes from the Benetton family who founded the company in 1965.»\n[2] «Scuderia | Scuderia means "stable" in the Italian language. It has entered English usage mainly through professional auto racing, in which many Italian teams incorporate the term in their names.»\n[3] «La Rinascente | La Rinascente (derived from Rinascita - means "rebirth" in Italian) is a Thai owned Italian retailer operating upscale department stores in the area of clothing, household and beauty products, founded in Milan in 1917 by Senatore Borletti from the